In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xlrd
import openpyxl

from config import *
from lib import *


# left_excel = os.path.join(STORAGE_DIR, 'tyr_en-chs.xlsx');
left_excel = os.path.join(TEMP_DIR,'Tyranny_string_result.xlsx')
right_excel = os.path.join(TEMP_DIR,'Tyranny_trans_result.xlsx')

ldf = pd.read_excel(left_excel).fillna('')
rdf = pd.read_excel(right_excel, index_col='index').fillna('')

ldf.index = [ldf['Name'] +'_'+ ldf['ID'].astype(str)]
ldf.index.name = 'index'


In [2]:
# xdf = ldf.join(rdf)
xdf = ldf

In [8]:
# 提取匹配字符
engloss = xdf['DefaultText'].str.extract('(\[url*?[^\]]*\].*?\[/url\])',expand=True).drop_duplicates().fillna('')
engloss
# cngloss = xdf['[ali]DefaultText'].str.extract('(\[(url*?)[^\]]*\].*?\[/url\])',expand=True).drop_duplicates().fillna('')
# cngloss
# cu = xdf['Custom'].str.extract('(\[(url*?)[^\]]*\].*?\[/url\])',expand=True).drop_duplicates().fillna('')
# cu

,0
index,
conversations\00_stronghold\00_cv_act2_skypillar_defense_arri_1,
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_1,[url=glossary:kyros]Kyros[/url]
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_2,[url=glossary:the voices of nerat]The Voices o...
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_4,[url=glossary:vendrien's well]Vendrien's Well[...
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_13,[url=glossary:tunon]Tunon[/url]
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_16,[url=glossary:ascension hall]Ascension Hall[/url]
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_31,[url=glossary:disfavored]Disfavored[/url]
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_32,[url=glossary:the voices of nerat]the Voices o...
conversations\00_stronghold\00_cv_act_2_alliescompanion_introduction_37,[url=glossary:edict]Edict[/url]


In [52]:
xdf = ldf.join(rdf)
for i,row in xdf.iterrows():
    if row['Custom'] == '':
        row['Custom'] = row['DefaultText']
    xdf.ix[i, 'Custom'] = row['Custom']

In [54]:
xdf.to_excel(os.path.join(TEMP_DIR, 'xdf.xlsx'))

In [4]:

list = []
for i,row in xdf.iterrows():
    x = {}
    x['Name'] = row['Name']
    x['ID'] = row['ID']
    if row['Custom']:
        x['DefaultText'] = row['Custom']
    elif row['[ali]DefaultText']:
        x['DefaultText'] = row['[ali]DefaultText']
    else:
        x['DefaultText'] = row['DefaultText']
    list.append(x)
    
ll = pd.DataFrame(list)

In [160]:
re = '(\[(url*?)[^\]]*\].*?\[/url\])'

lx = ll['DefaultText'].str.extract('(\[(url*?)[^\]]*\].*?\[/url\])',expand=True)

In [161]:
la = lx.drop_duplicates()

In [11]:
quo = xdf['DefaultText'].str.extract('(\[[^url/url][^\]]*\])',expand=True).drop_duplicates().fillna('')

In [12]:
quo.to_excel(os.path.join(TEMP_DIR, 'quo.xlsx'))

In [43]:
# e.g 
# [url=glossary:blinded]Blinded[/url] keyilo [url=glossary:blinded]目盲[/url]

# \[url=glossary:blinded.*?(?=\])]
# >> [url=glossary:blinded]

# \[(url*?)[^\]]*\].*?\[/url\]
# >> [url=glossary:blinded]Blinded[/url] || [url=glossary:blinded]目盲[/url]

gloss = os.path.join(STORAGE_DIR, 'tyr_gloss.xlsx')
gl = pd.read_excel(gloss)
gl = gl.join(gl[0].str.extract('(\[url=(.*?[^\]]*)\].*?\[/url\])',expand=True)[1], how='outer')
gl

,0,cus0,1
0,[url=glossary:ability cooldown]Ability Cooldow...,[url=glossary:ability cooldown]技能冷却[/url],glossary:ability cooldown
1,[url=glossary:accessory]Accessory[/url],[url=glossary:accessory]饰品[/url],glossary:accessory
2,[url=glossary:accuracy]Accuracy[/url],[url=glossary:accuracy]精准度[/url],glossary:accuracy
3,[url=glossary:affliction]Affliction[/url],[url=glossary:affliction]负面状态[/url],glossary:affliction
4,[url=glossary:armor penetration]Armor Penetrat...,[url=glossary:armor penetration]护甲穿透[/url],glossary:armor penetration
5,[url=glossary:armor]Armor[/url],[url=glossary:armor]护甲[/url],glossary:armor
6,[url=glossary:ascension hall]Ascension Hall[/url],[url=glossary:ascension hall]升灵堂[/url],glossary:ascension hall
7,[url=glossary:asleep]Asleep[/url],[url=glossary:asleep]沉睡[/url],glossary:asleep
8,[url=glossary:athletics]Athletics[/url],[url=glossary:athletics]运动[/url],glossary:athletics
9,[url=glossary:bastard city]Bastard City[/url],[url=glossary:bastard city]恶棍城[/url],glossary:bastard city
